1. Ordinary Least Squares or Weighted Least Squares?
Consider a dataset generated by the following model: For $i \in \{1, \dots, n\},$

$$Y_i = 5 X_i + \varepsilon_i,$$

where $X_i$ follows a uniform distribution between 1 and 10 and $\varepsilon_i$ is an error whose conditional distribution given $X_i =x_i$ is a Gaussian distribution with the zero mean and the variance $0.01 \times x_i^2$. With conventional notation,
$$
X_i \sim U[1,10], \\
\varepsilon_i \mid X_i = x_i \sim N(0, 0.01 x_i ^2).
$$
Throughout this problem, we will assume that the intercept is always zero.


1-0.
Write a function called `simulate_data` that takes as input a sample size `n` and outputs two objects `x` and `y`. Both `x` and `y` should be 1D `numpy.ndarray` of the shape $(n,)$.
 - For uniform number generation, you can use `numpy.random.uniform`. Just for your reference, it takes as input the three parameters `(low, high, size)`.
 - For Gaussian number generation, you can use `numpy.random.normal`. It takes as input the three parameters `(loc, scale, size)`.


In [2]:
import numpy as np
def simulate_data(n:int):
  x = np.random.uniform(low= 1, high=10, size=n)
  error = (0.1*x)*np.random.normal(size=n)
  y = 5 * x + error
  return x, y

1-1.
Write a function `ols_estimate` that computes ordinary least squares (OLS). Your function should take as input `x` and `y`. Similarly, write a function `wls_estimate` that computes weighted least squares (WLS) regressions. You can use the following closed-form solutions:

$$
\beta_{OLS} = \frac{\sum_{i=1}^n x_i y_i }{\sum_{i=1}^n x_i ^2 }, \quad \beta_{WLS} = \frac{\sum_{i=1}^n w_i x_i y_i }{\sum_{i=1}^n w_i  x_i ^2 },
$$
where $w_i = 1 / (0.1 \times x_i)^2$.

In [3]:
def ols_estimate(x, y):
  return np.sum(x*y)/np.sum(x*x)
def wls_estimate(x, y):
  w = 1/(0.1*x)**2
  return np.sum(w*x*y)/np.sum(w*x*x)

1-2.
Using functions created in steps [Q0-0] and [Q0-1], conduct the following simulation study.
 - Generate a dataset with `n=100` and compute OLS and WLS estimates.
 - Repeat this process `num_trials=500` times and stack your estimates to a list. The name of each method's estimates should be `ols_mse` and `wls_mse`.
 - For each method, compute the mean squared error of the estimated slope $\beta$. The name of each method's MSE should be `mean_ols_mse` and `mean_wls_mse`.
 - We denote the $j$-th estimated slope by  $\hat{\beta}_j$. Then, MSE of an estimator is given as follows.
 $$ MSE( \hat{\beta}_1, \dots, \hat{\beta}_{N} ) = \frac{1}{N} \sum_{j=1} ^N ( \hat{\beta}_j - \beta)^2 .$$


where $\beta=5$ is the true slope, and $N=500$ denotes `num_trials`.
 - Which methods provides a smaller MSE? It should be `WLS`.

In [4]:
num_trials=500
n=100
ols_list, wls_list =[], []
for trial_ind in range(num_trials):
  #generate data
  x, y = simulate_data(n)

  #stack ols/wls estimates
  ols_list.append(ols_estimate(x,y))
  wls_list.append(wls_estimate(x,y))

mean_ols_mse = np.mean((np.array(ols_list)-5)**2)
mean_wls_mse = np.mean((np.array(wls_list)-5)**2)
mean_ols_mse, mean_wls_mse

(np.float64(0.00016650012168380646), np.float64(9.9949764473581e-05))

In [5]:
x, y = simulate_data(n)
x_mat = x.reshape(-1,1)
W= np.diag(1/(0.01*(x**2)))
#W = np.linalg.inv(0.01*x_mat.dot(x_mat.T))
#print(x.shape, x_mat.shape, y.shape, W.shape)

In [6]:
#bread_part = (x_mat.T.dot(x_mat)).dot(x_mat.T) #(1,n)
bread_part = np.sum(x**2)*(x_mat)
ham_part = 1/float(x_mat.T.dot(W).dot(x_mat))
bread_part.shape, ham_part

/tmp/ipython-input-2388925528.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ham_part = 1/float(x_mat.T.dot(W).dot(x_mat))


((100, 1), 0.0001)

1-3.
Next, we will repeat [Q0-2] for different sample sizes and store the results in a `pandas.DataFrame` named `df_dict`.
- `df_dict` consists of two rows, representing the OLS and WLS methods.
- It has five columns: the first column specifies the method (`OLS` or `WLS`, a string value), and the remaining four columns correspond to sample sizes `(10, 60, 110, 160)`.
The output is expected to be similar to the following table.
```
method	10	60	110	160
0	OLS	0.001703	0.000298	0.000149	0.000112
1	WLS	0.001030	0.000164	0.000097	0.000064
```

In [7]:
def helper(n:int):
  num_trials=500
  ols_list, wls_list =[], []
  for trial_ind in range(num_trials):
  #generate data
    x, y = simulate_data(n)

  #stack ols/wls estimates
    ols_list.append(ols_estimate(x,y))
    wls_list.append(wls_estimate(x,y))

  mean_ols_mse = np.mean((np.array(ols_list)-5)**2)
  mean_wls_mse = np.mean((np.array(wls_list)-5)**2)
  return mean_ols_mse, mean_wls_mse

In [8]:
import pandas as pd
ols_dict={"method":"OLS"}
wls_dict={"method":"WLS"}

for sample_size in [10,60,11,160]:
  mean_ols_mse, mean_wls_mse = helper(sample_size)
  ols_dict[sample_size] = mean_ols_mse
  wls_dict[sample_size] = mean_wls_mse
df_dict=pd.DataFrame([ols_dict, wls_dict])
df_dict

,method,10,60,11,160
0,OLS,0.001657,0.000269,0.001502,0.000101
1,WLS,0.001050,0.000170,0.000934,0.000062


In [13]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/earthquake_query.csv'

df_earthquake_raw=pd.read_csv(file_path)
df_earthquake_raw

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,time,latitude,longitude,depth,mag,magType,gap,dmin,rms,net,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-12-31T03:31:43.830Z,-23.1036,-68.8969,96.224,5.1,mww,42.0,0.679,1.05,us,...,2023-01-22T05:50:40.700Z,"Antofagasta, Chile",earthquake,4.87,3.429,0.052,36.0,reviewed,us,us
1,2022-12-30T21:53:29.589Z,-14.9955,-75.7501,10.338,5.2,mww,104.0,3.173,0.61,us,...,2023-02-03T02:36:19.040Z,"67 km WSW of Changuillo, Peru",earthquake,8.14,3.495,0.069,20.0,reviewed,us,us
2,2022-12-30T21:02:26.831Z,-52.7091,27.9903,10.000,5.2,mww,37.0,16.666,0.64,us,...,2023-02-03T02:32:32.040Z,south of Africa,earthquake,11.59,1.786,0.098,10.0,reviewed,us,us
3,2022-12-30T12:10:45.211Z,-7.9548,128.8795,35.000,5.0,mb,56.0,2.396,0.54,us,...,2023-01-14T10:36:17.040Z,"216 km ENE of Lospalos, Timor Leste",earthquake,3.41,1.898,0.067,71.0,reviewed,us,us
4,2022-12-30T09:42:09.278Z,-21.8971,-174.7611,22.640,5.4,mww,45.0,0.833,0.67,us,...,2023-01-26T12:04:30.322Z,"65 km SSE of ‘Ohonua, Tonga",earthquake,7.45,4.744,0.083,14.0,reviewed,us,us
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12052,2016-01-02T08:37:22.510Z,36.5426,70.9320,189.000,5.2,mwb,58.0,0.648,0.91,us,...,2022-08-02T02:25:09.883Z,"36 km SSE of Jurm, Afghanistan",earthquake,5.20,1.600,0.049,40.0,reviewed,us,us
12053,2016-01-02T04:22:19.570Z,44.8069,129.9406,585.470,5.8,mww,12.0,0.311,0.89,us,...,2022-08-02T02:21:30.587Z,"21 km ENE of Chaihe, China",earthquake,8.10,3.600,NaN,NaN,reviewed,us,us
12054,2016-01-01T23:04:52.280Z,-20.5653,-179.1565,642.750,5.0,mb,91.0,3.850,0.83,us,...,2016-11-10T22:06:53.351Z,Fiji region,earthquake,10.70,5.800,0.033,298.0,reviewed,us,us
12055,2016-01-01T15:02:16.740Z,-28.6278,-177.2810,34.000,5.8,mww,62.0,0.837,1.28,us,...,2022-05-03T19:18:45.242Z,Kermadec Islands region,earthquake,7.80,1.200,NaN,NaN,reviewed,us,us


2-1.
The `df_earthquake_raw` includes some missing data (a data with at least one missing cell). Remove all the missing data in `df_earthquake_raw`.
 - After removing missing data, please store the completely observed dataset with the name `df_earthquake_complete`.
 - The index of `df_earthquake_complete` should be reset after the removal so that indices are well-ordered numbers. Also, the original index column should be dropped.

In [14]:
df_earthquake_complete = df_earthquake_raw.dropna().reset_index(drop=True)
df_earthquake_complete

,time,latitude,longitude,depth,mag,magType,gap,dmin,rms,net,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-12-31T03:31:43.830Z,-23.1036,-68.8969,96.224,5.1,mww,42.0,0.679,1.05,us,...,2023-01-22T05:50:40.700Z,"Antofagasta, Chile",earthquake,4.87,3.429,0.052,36.0,reviewed,us,us
1,2022-12-30T21:53:29.589Z,-14.9955,-75.7501,10.338,5.2,mww,104.0,3.173,0.61,us,...,2023-02-03T02:36:19.040Z,"67 km WSW of Changuillo, Peru",earthquake,8.14,3.495,0.069,20.0,reviewed,us,us
2,2022-12-30T21:02:26.831Z,-52.7091,27.9903,10.000,5.2,mww,37.0,16.666,0.64,us,...,2023-02-03T02:32:32.040Z,south of Africa,earthquake,11.59,1.786,0.098,10.0,reviewed,us,us
3,2022-12-30T12:10:45.211Z,-7.9548,128.8795,35.000,5.0,mb,56.0,2.396,0.54,us,...,2023-01-14T10:36:17.040Z,"216 km ENE of Lospalos, Timor Leste",earthquake,3.41,1.898,0.067,71.0,reviewed,us,us
4,2022-12-30T09:42:09.278Z,-21.8971,-174.7611,22.640,5.4,mww,45.0,0.833,0.67,us,...,2023-01-26T12:04:30.322Z,"65 km SSE of ‘Ohonua, Tonga",earthquake,7.45,4.744,0.083,14.0,reviewed,us,us
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,2016-01-05T05:28:30.270Z,-5.9576,130.5984,129.370,5.2,mb,26.0,2.128,0.91,us,...,2016-11-10T22:07:09.165Z,"241 km W of Tual, Indonesia",earthquake,7.00,5.400,0.062,86.0,reviewed,us,us
11082,2016-01-05T01:58:19.580Z,15.3798,-95.3519,19.100,5.0,mb,80.0,1.759,0.80,us,...,2022-08-02T02:55:59.077Z,"75 km SSE of Santiago Astata, Mexico",earthquake,5.40,4.200,0.030,347.0,reviewed,us,us
11083,2016-01-04T14:36:13.490Z,19.1416,121.0647,10.000,5.2,mb,77.0,3.657,0.64,us,...,2022-08-02T02:49:06.561Z,"59 km N of Claveria, Philippines",earthquake,7.30,1.800,0.048,143.0,reviewed,us,us
11084,2016-01-02T08:37:22.510Z,36.5426,70.9320,189.000,5.2,mwb,58.0,0.648,0.91,us,...,2022-08-02T02:25:09.883Z,"36 km SSE of Jurm, Afghanistan",earthquake,5.20,1.600,0.049,40.0,reviewed,us,us


2-2.
Print out a human readable message that explains the shape of `df_earthquake_complete` using `f-strings`.
 - Please use a built-in function `print()`.
 - Please do NOT use a hard-coding.
 - Your output should look like
```
df_earthquake_complete has 11086 rows and 21 columns.
```

In [15]:
print(f"df_earthquake_complete has {df_earthquake_complete.shape[0]}  rows and {df_earthquake_complete.shape[1]}  columns.")

df_earthquake_complete has 11086  rows and 21  columns.


2-3.
Calculate the proportion of the missing data in `df_earthquake_raw` and store it with the name `missing_rate`. The value should be greater than 0 and smaller than 1. And print out `missing_rate` in a human readable message.

The output should be as follows.
```
The missing rate is 0.08053412955129802
```

In [18]:
missing_rate = 1 - len(df_earthquake_complete)/len(df_earthquake_raw)
print(f"The missing rate is {missing_rate}")

The missing rate is 0.08053412955129802


2-4.
Check whether there are any duplicated data in `df_earthquake_complete`. We will implement this with a if/else control flow statement.
 - First store an `int` variable `number_of_duplicate` that has the total number of duplicates.
 - (if block) If there are duplicates, remove every duplicates except the one with the lowest index number.
 - (else block) If there are no duplicates, please print out `No duplicates!`.

In [19]:
number_of_duplicate = int(df_earthquake_complete.duplicated().sum())
if number_of_duplicate != 0:
  df_earthquake_complete = df_earthquake_complete.drop_duplicates(keep="first").reset_index()
else:
  print("No duplicates!")

No duplicates!


2-5.
Add a new column called `region` to `df_earthquake_complete` that describes a region of the column `place`. We define a region of a `place` as follows.
 - if `place` does not include a comma `,`, then `place` itself becomes a region.
   - if `place` is `south of Africa`, then its region is `south of Africa`.
 - if `place` includes one or multiple commas `,`, the text after the last `,`
   - if `place` is `Antofagasta, Chile`, then its region is `Chile`.
 - The region data should not start with a whitespace (Hint: `str.strip()`).

In [20]:

df_earthquake_complete["region"] = (
  df_earthquake_complete["place"]
  .apply(lambda x: x.split(",")[-1].strip())
)

2-6.
Add a new column called `year` to `df_earthquake_complete` that describes the year that earthquake occured.
 - You can extract the year from the column `time`.
 - The type of `year` must be `int`.

In [21]:
df_earthquake_complete["year"] = (
  df_earthquake_complete["time"]
  .apply(lambda x: int(x[:4]))
  )

2-7.
Create a new `pandas.DataFrame` called `df_earthquake` by selecting a subset of columns from `df_earthquake_complete`
 - A list of columns we are interested in is  `['time','year', 'depth', 'mag', 'region', 'depthError', 'magError']`
 - Take every rows from `df_earthquake_complete`.


In [22]:
list_of_columns=['time','year', 'depth', 'mag', 'region', 'depthError', 'magError']
df_earthquake = df_earthquake_complete.loc[:, list_of_columns]

2-8.
Create a list called `earthquake_prone_region` containing the 5 most earthquake-prone regions.
 - Expected outcome:
 ```
 ['Indonesia', 'Papua New Guinea', 'South Sandwich Islands region', 'Tonga', 'Japan']
 ```

In [25]:
earthquake_prone_region = (
  df_earthquake['region'].value_counts()
  .reset_index().iloc[:5]
  .loc[:, "region"].to_list()
)

2-9.

Create a list called `major_earthquake_region` containing regions with the top 5 average magnitudes.
 - Magnitude of the earthquake is defined in the `mag` column.
 - Expected outcome:
 ```
 ['Jamaica', 'Flores Sea', 'Java Sea', 'Brazil', 'North Pacific Ocean']
 ```


In [28]:
major_earthquake_region = (
  df_earthquake.groupby("region")["mag"].mean()
  .reset_index()
  .rename(columns={"mag":"avg_mag"})
  .sort_values(by="avg_mag", ascending=False)
  .iloc[:5].loc[:,"region"].to_list()
)

2-10.
Create a `pandas.DataFrame` called `df_region_year` by summarizing data from `df_earthquake` as follows.
 - Include all unique (`region`, `year`) pairs from df_earthquake.
 - The `df_region_year` should have three columns, `region`, `year`, and `mean_mag`. The column `mean_mag` is defined as the average `mag` for each (`region`, `year`) pair.
 - Sort the rows in non-ascending order based on `mean_mag`.
 - Expected outcome of `df_region_year.iloc[0]` is
  ```
  region      Jamaica
  year           2020
  mean_mag        7.7
  ```


In [31]:
df_region_year = (
  df_earthquake.groupby(["region", "year"])["mag"].mean()
  .reset_index()
  .rename(columns={"mag":"mean_mag"})
  .sort_values(by="mean_mag", ascending=False).reset_index(drop=True) #drop the original
)
df_region_year.iloc[0]

,0
region,Jamaica
year,2020
mean_mag,7.7
